In [51]:
import utils 
import csv 
import pandas as pd

text_files, json_files = utils.return_file_path()
df = pd.DataFrame(columns=["Qustion", "Output", "Predict", "Answer", "Difficulty" ,"Number"])

for text_file, json_file in zip(text_files, json_files):
    output = "" 
    start_idx = 0 
    
    answer = ("prime" if "prime" in text_file else "composite")
    data_type = json_file.rstrip(".json").lstrip("../json_sampled/")
    querys = utils.return_json_data(json_file) 
    
    for query in querys:
        query_list = list(query.values())
        for i in range(len(query_list)-2):
            output, start_idx = utils.return_output(text_file, start_idx)
            if i >= 4:
                continue
            result = utils.return_PrimeOrComposite(output, i)
            
            # print(query["number"], f"Q{i+1}", output, result, query["answer"])
            df.loc[str(query["number"])+f"_Q{i+1}"] = (f"Q{i+1}", output, result, query["answer"], data_type ,query["number"])
df

,Qustion,Output,Predict,Answer,Difficulty,Number
10_Q1,Q1,\nno\n,composite,composite,2_digit_composite,10
10_Q2,Q2,\n10 is divisible by 2 and 5. So 10 is not a p...,composite,composite,2_digit_composite,10
10_Q3,Q3,\nyes\n,composite,composite,2_digit_composite,10
10_Q4,Q4,\n10 is divisible by 2 and 5. 10 is a composit...,composite,composite,2_digit_composite,10
12_Q1,Q1,\nno\n,composite,composite,2_digit_composite,12
...,...,...,...,...,...,...
2939_Q4,Q4,\n2939 is divisible by 3 because the sum of it...,composite,prime,4_digit_prime,2939
6521_Q1,Q1,\nno\n,composite,prime,4_digit_prime,6521
6521_Q2,Q2,\n6521 is divisible by 3 because the sum of it...,composite,prime,4_digit_prime,6521
6521_Q3,Q3,\nyes\n,composite,prime,4_digit_prime,6521


In [208]:
df.loc['8797_Q4', 'Output']

'\n8797 is divisible by 3 because the sum of its digits is 21, which is divisible by 3. So, the final answer is yes.\n'

In [52]:
df.groupby('Difficulty').count()

,Qustion,Output,Predict,Answer,Number
Difficulty,,,,,
2_digit_composite,276,276,276,276,276
2_digit_prime,84,84,84,84,84
3_digit_easy_composite,808,808,808,808,808
3_digit_medium_composite,224,224,224,224,224
3_digit_prime,572,572,572,572,572
4_digit_easy_composite,2000,2000,2000,2000,2000
4_digit_hard_composite,524,524,524,524,524
4_digit_medium_composite,2000,2000,2000,2000,2000
4_digit_prime,2000,2000,2000,2000,2000


In [53]:
df_idk = df[df["Predict"]=="idk"]
df_idk.to_csv("../csv/PaLM2_idk.csv")

In [54]:
import random 

random.seed(1024)

sampled_df = pd.DataFrame(columns=["Qustion", "Output", "Predict", "Answer", "Difficulty" ,"Number"])

Difficulty_list = df['Difficulty'].unique()
for Difficulty in Difficulty_list:
    temp_df = df[df['Difficulty'] == Difficulty]
    number_list = list(temp_df['Number'].unique())

    if len(number_list) >= 25:
        number_list = random.sample(number_list, k=25)
    
    for number in number_list:    
        df_25 = temp_df[temp_df['Number']==number]
        sampled_df = pd.concat([sampled_df, df_25])

sampled_df = sampled_df.drop(["Qustion", "Answer", "Difficulty", "Number"], axis=1)
sampled_df['Verdict'] = 0
# sampled_df.to_csv("../csv/PaLM2_Sampled.csv")

In [55]:
len(sampled_df[sampled_df['Predict']=='idk'])

48

In [56]:
import numpy as np

acc_df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4"])

Difficulty_list = df['Difficulty'].unique()
for Difficulty in Difficulty_list:
    if Difficulty[-9:] == "composite":
        continue
    acc = np.array([0, 0, 0, 0])
    temp_df = df[df['Difficulty'] == Difficulty]

    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        if row['Predict'] == row['Answer']:
            acc[i] += 1
    acc = acc / (len(temp_df) // 4) * 100
    
    acc_df.loc[Difficulty] = acc
acc_df.round(2)

,Q1,Q2,Q3,Q4
2_digit_prime,100.00,61.9,66.67,80.95
3_digit_prime,57.34,2.1,0.70,6.29
4_digit_prime,3.60,0.2,0.00,0.00


In [238]:
acc_df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4"])

Difficulty_list = df['Difficulty'].unique()
acc = np.array([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], dtype="float")
digit_len = np.array([0.0, 0.0, 0.0])

composite = 0

for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue    
    digit = int(Difficulty[0])
    temp_df = df[df['Difficulty'] == Difficulty]
    digit_len[digit-2] += (len(temp_df) // 4)
    
    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        if row['Predict'] == row['Answer']:
            acc[digit-2][i] += 1
            composite += 1
            
for i in range(3):
    # acc[i] = acc[i] / digit_len[i] * 100
    acc_df.loc[f"{i+2}_digit_composite"] = acc[i]

print(composite, "/8488")

acc_df.round(2)

5614 /8488


,Q1,Q2,Q3,Q4
2_digit_composite,67.0,57.0,69.0,69.0
3_digit_composite,193.0,206.0,258.0,231.0
4_digit_composite,1081.0,1124.0,1131.0,1128.0


In [240]:
acc_df.sum().sum() - 342

5272.0

In [58]:
acc_df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4"])

Difficulty_list = df['Difficulty'].unique()
for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue
    
    acc = np.array([0, 0, 0, 0])
    temp_df = df[df['Difficulty'] == Difficulty]

    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        if row['Predict'] == row['Answer']:
            acc[i] += 1
    acc = acc / (len(temp_df) // 4) * 100
    
    acc_df.loc[Difficulty] = acc
acc_df.round(2)

,Q1,Q2,Q3,Q4
2_digit_composite,97.10,82.61,100.0,100.00
3_digit_easy_composite,80.69,81.68,100.0,90.59
3_digit_medium_composite,53.57,73.21,100.0,85.71
4_digit_easy_composite,98.80,99.60,100.0,100.00
4_digit_hard_composite,93.13,100.00,100.0,100.00
4_digit_medium_composite,93.00,99.00,100.0,99.40


In [73]:
acc_df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4"])
cnt_df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4"])
Difficulty_list = df['Difficulty'].unique()
for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue
    cnt = np.array([0, 0, 0, 0])
    acc = np.array([0, 0, 0, 0])
    temp_df = df[df['Difficulty'] == Difficulty]

    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        cnt[i] += 1
        if row['Predict'] == row['Answer']:
            acc[i] += 1
    # acc = acc / (len(temp_df) // 4) * 100
    cnt_df.loc[Difficulty] = cnt
    acc_df.loc[Difficulty] = acc
acc_df.round(2)

,Q1,Q2,Q3,Q4
2_digit_composite,67,57,69,69
3_digit_easy_composite,163,165,202,183
3_digit_medium_composite,30,41,56,48
4_digit_easy_composite,494,498,500,500
4_digit_hard_composite,122,131,131,131
4_digit_medium_composite,465,495,500,497


In [107]:
# 2 ~ 10000 : prime number
n = 10000
a = [False, False] + [True] * (n-1)
primes = [] 

for i in range(2, n+1):
  if a[i]:
    primes.append(i)
    for j in range(2*i, n+1, i):
      a[j] = False 
primes[:5]

[2, 3, 5, 7, 11]

In [228]:
pd.set_option('mode.chained_assignment',  None)

      
Difficulty_list = df['Difficulty'].unique()

undefined_df = pd.DataFrame(columns=["Qustion", "Output", "Predict", "Answer", "Difficulty" ,"Number", "Factor1", "Factor2"])
composite_cnt = 0

err_df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4"])


Difficulty_list = df['Difficulty'].unique()

right_composite = 0
wrong_composite = 0
for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue
    
    acc = np.array([0, 0, 0, 0])
    wrong_patton_acc = np.array([0, 0, 0, 0])
    
    temp_df = df[df['Difficulty'] == Difficulty]

    for idx, row in temp_df.iterrows():        
        i = int(idx[-1])-1
        if row['Predict'] == row['Answer']:
            acc[i] += 1
            if i==1 or i==3:
                right_composite += 1
        else: 
            continue
        
        # if f"{row['Number']} is divisible by 2, so it is not a prime number." in row['Output']:
        #     continue
        # if f"{row['Number']} is divisible by 2, so it is not prime." in  row['Output']:
        #     continue
        fast_cut = False
        
        number_list = [3, 7, 11, 2, 5, 13, 17]
        for wrong_number in number_list:
            if (row['Number']%wrong_number)!= 0:
                s= f"{row['Number']} is divisible by {wrong_number}"
                if  row['Output'].find(s) != -1:
                    wrong_patton_acc[i] += 1
                    wrong_composite += 1
                    fast_cut = True
                    break
                number_list = [3, 7, 11, 2, 5, 13, 17]
        
        if fast_cut:
            continue                
        
        for wrong_number in number_list:
            if (row['Number']%wrong_number)!= 0:
                s= f"{row['Number']} is also divisible by {wrong_number}"
                if  row['Output'].find(s) != -1:
                    wrong_patton_acc[i] += 1
                    wrong_composite += 1
                    fast_cut = True
                    break
            
        if fast_cut:
            continue
        
        if row['Output'].rfind("is divisible by") != -1:
            cut_output = row['Output'][row['Output'].rfind("is divisible by")+len("is divisible by")+1:]
            cut_output = cut_output.split(".")[0]
            cut_output = cut_output.replace("and", ",")
            cut_output = cut_output.replace(" , ", "") 
            cut_output = cut_output.replace("itself", f"{row['Number']}")
            
            
            try: 
                num_list = list(map(int, cut_output.split(",")))
            except:
                if len(cut_output) >= 3:
                    if cut_output[-3] == ",":
                        cut_output = cut_output[:-5]
                        num_list = list(map(int, cut_output.split(",")))
            
            try:
                # num_list = list(map(int, cut_output.split(",")))
                for factor in num_list:
                    if (row['Number']%factor)==0:
                        continue
                    else:
                        fast_cut = True
                        wrong_composite += 1
                        break 
            except:
                if i==1 or i==3:
                    undefined_df.loc[f"{row['Number']}_Q{i+1}"] = row  
    
        if fast_cut:
            continue 
                    
        if i==1 or i==3:
            # Factors
            for prime in primes:
                if (row['Number']%prime) == 0:
                    factor1 = prime 
                    factor2 = row['Number']//prime
                    break
            undefined_df.loc[f"{row['Number']}_Q{i+1}"] = row
            undefined_df.loc[f"{row['Number']}_Q{i+1}", 'Factor1'] = factor1
            undefined_df.loc[f"{row['Number']}_Q{i+1}", 'Factor2'] = factor2
            
    # err_df.loc[Difficulty] = wrong_patton_acc / acc * 100
    err_df.loc[Difficulty] = wrong_patton_acc
    
    
# err_df = err_df.drop(["Q1", "Q3"], axis=1)
err_df.round(2).reindex(['2_digit_composite',
                        '3_digit_easy_composite', 
                        '4_digit_easy_composite',
                        '3_digit_medium_composite',
                        '4_digit_medium_composite', 
                        '4_digit_hard_composite'])

,Q1,Q2,Q3,Q4
2_digit_composite,0,0,0,3
3_digit_easy_composite,0,60,0,83
4_digit_easy_composite,0,301,0,305
3_digit_medium_composite,0,38,0,47
4_digit_medium_composite,0,436,0,487
4_digit_hard_composite,0,124,0,131


In [172]:
undefined_df['Verdict'] = "correct"
# undefined_df.to_csv("../csv/undefined.csv")
undefined_df


,Qustion,Output,Predict,Answer,Difficulty,Number,Factor1,Factor2,Verdict
12_Q2,Q2,"\n12 is divisible by 2, 3, 4, 6, and 12. So 12...",composite,composite,2_digit_composite,12,2,6,correct
12_Q4,Q4,"\n12 is divisible by 2, 3, 4, 6, and 12. So 12...",composite,composite,2_digit_composite,12,2,6,correct
15_Q2,Q2,\n15 is divisible by 3. 15 is not a prime numb...,composite,composite,2_digit_composite,15,3,5,correct
16_Q2,Q2,"\n16 is divisible by 2, 4, 8, and 16. So 16 is...",composite,composite,2_digit_composite,16,2,8,correct
16_Q4,Q4,"\n16 is divisible by 2, 4, 8, and 16. 16 is a ...",composite,composite,2_digit_composite,16,2,8,correct
...,...,...,...,...,...,...,...,...,...
1111_Q4,Q4,\n1111 is divisible by 11. 1111 is a composite...,composite,composite,4_digit_medium_composite,1111,11,101,correct
4411_Q2,Q2,\n4411 is divisible by 11. 4411 is not a prime...,composite,composite,4_digit_medium_composite,4411,11,401,correct
1751_Q2,Q2,\n1751 is divisible by 17. 1751 is not a prime...,composite,composite,4_digit_medium_composite,1751,17,103,correct
1751_Q4,Q4,\n1751 is divisible by 17. 1751 is a composite...,composite,composite,4_digit_medium_composite,1751,17,103,correct


In [192]:
err2_df = pd.read_csv("../csv/undefined.csv")
err_cnt = {}
for idx, row in err2_df.iterrows():
    if row['Verdict'] == "wrong":
        try: 
            err_cnt[row['Difficulty']+"_"+row['Qustion']] += 1
        except: 
            err_cnt[row['Difficulty']+"_"+row['Qustion']] = 1
err_cnt

{'3_digit_easy_composite_Q4': 1}

In [230]:
err2_df[err2_df['Verdict']==
        'wrong']

,Unnamed: 0,Qustion,Output,Predict,Answer,Difficulty,Number,Factor1,Factor2,Verdict
120,301_Q4,Q4,\n301 is not prime because 301 = 13 * 23.\nThu...,composite,composite,3_digit_easy_composite,301,7,43,wrong


In [202]:
err_df.loc['3_digit_easy_composite', 'Q4'] += 1

In [204]:
(err_df / cnt_df * 100).round(2).reindex(['2_digit_composite',
                                                   '3_digit_easy_composite', 
                                                   '4_digit_easy_composite',
                                                   '3_digit_medium_composite',
                                                   '4_digit_medium_composite', 
                                                   '4_digit_hard_composite'])

,Q1,Q2,Q3,Q4
2_digit_composite,0.0,0.00,0.0,4.35
3_digit_easy_composite,0.0,29.70,0.0,41.58
4_digit_easy_composite,0.0,60.20,0.0,61.00
3_digit_medium_composite,0.0,67.86,0.0,83.93
4_digit_medium_composite,0.0,87.20,0.0,97.40
4_digit_hard_composite,0.0,94.66,0.0,100.00


In [209]:
((acc_df-err_df) / cnt_df * 100).round(2).reindex(['2_digit_composite',
                                                   '3_digit_easy_composite', 
                                                   '4_digit_easy_composite',
                                                   '3_digit_medium_composite',
                                                   '4_digit_medium_composite', 
                                                   '4_digit_hard_composite']).to_csv("err_palm2.csv")

In [169]:
undefined_df["Verdict"] = 0
print(len(undefined_df))
# undefined_df.to_csv("../csv/undefined.csv")

342


In [62]:
''' 
554 is divisible by 2 because 554 / 2 = 277. 277 is divisible by 3 because 277 / 3 = 92. 92 is divisible by 2 because 92 / 2 = 46. 46 is divisible by 2 because 46 / 2 = 23. 23 is a prime number. So 554 is not a prime number.
"
785 is divisible by 5 because the last digit is 5. 785 is divisible by 19 because 785 / 19 = 41. 785 is a composite number.
"
"
965 is divisible by 5 because the last digit is 5. 965 is divisible by 19 because 965 / 19 = 50. 965 is a composite number.
"
"
3153 is divisible by 3 because the sum of its digits is 3 + 1 + 5 + 3 = 12, which is divisible by 3. 3153 is also divisible by 7 because 3153 / 7 = 450.5, which is rounded up to 451. 451 is divisible by 7 because 451 / 7 = 64.42857142857143, which is rounded up to 65. 6
"
"
3603 is divisible by 3 because the sum of its digits is 3 + 6 + 0 + 3 = 12, which is divisible by 3. 3603 is also divisible by 7 because 3603 / 7 = 514.57142857142857. Since 3603 is divisible by both 3 and 7, it is a composite number.
"
"
2798 is divisible by 2. 2798 / 2 = 1399. 1399 is divisible by 3. 1399 / 3 = 466. 466 is divisible by 2. 466 / 2 = 233. 233 is divisible by 3. 233 / 3 = 77. 77 is divisible by 3. 77 / 3 = 25. 25 is divisible by 
"
"
985 is divisible by 5 because the last digit is 5. 985 is divisible by 19 because 985 / 19 = 51. 985 is a composite number.
"
"
9149 is divisible by 7 because 9149 / 7 = 1307. 1307 is divisible by 7 because 1307 / 7 = 186. 186 is divisible by 7 because 186 / 7 = 26. 26 is divisible by 7 because 26 / 7 = 3. 3 is not divisible by 7. So 9149 is not a prime number.
"

'''

' \n554 is divisible by 2 because 554 / 2 = 277. 277 is divisible by 3 because 277 / 3 = 92. 92 is divisible by 2 because 92 / 2 = 46. 46 is divisible by 2 because 46 / 2 = 23. 23 is a prime number. So 554 is not a prime number.\n"\n785 is divisible by 5 because the last digit is 5. 785 is divisible by 19 because 785 / 19 = 41. 785 is a composite number.\n"\n"\n965 is divisible by 5 because the last digit is 5. 965 is divisible by 19 because 965 / 19 = 50. 965 is a composite number.\n"\n"\n3153 is divisible by 3 because the sum of its digits is 3 + 1 + 5 + 3 = 12, which is divisible by 3. 3153 is also divisible by 7 because 3153 / 7 = 450.5, which is rounded up to 451. 451 is divisible by 7 because 451 / 7 = 64.42857142857143, which is rounded up to 65. 6\n"\n"\n3603 is divisible by 3 because the sum of its digits is 3 + 6 + 0 + 3 = 12, which is divisible by 3. 3603 is also divisible by 7 because 3603 / 7 = 514.57142857142857. Since 3603 is divisible by both 3 and 7, it is a composite 

In [63]:
## 배수 판정법에 기반한 방법: https://ko.wikipedia.org/wiki/%EB%B0%B0%EC%88%98_%ED%8C%90%EC%A0%95%EB%B2%95#:~:text=%EB%8B%A4%EC%9D%8C%EC%9D%80%2043%EA%B0%80%EC%A7%80%20%EC%9E%90%EC%97%B0%EC%88%98,%EC%9D%98%20%EB%B0%B0%EC%88%98%EC%9D%B8%20%EC%88%98%EC%9D%B4%EB%8B%A4.
## 2_digit_composite 에러 케이스 정리 
# 56 is divisible by 2, 4, 8, 16, and 32
# 58 is divisible by 2, 7, and 1. So 58 is not a prime number.
# 68 is divisible by 2, 4, 7, and 14. So 68 is not a prime number.
# 76 is divisible by 2, 4, 19, and 38. So 76 is not a prime number.
# 77 is divisible by 3 because the sum of its digits is 7 + 7 = 14, which is divisible by 3. So 77 is a composite number.
# 82 is divisible by 2, 7, and 14. 82 is a composite number.
# 84 is divisible by 2, 3, 4, 6, 9, 12, 18, 24, 36, 42, and 84. So 84 is not a prime number.
# 84 is divisible by 2, 3, 4, 6, 9, 12, 18, 27, 36, 42, 54, 84. So 84 is a composite number.
# 86 is divisible by 2, 7, and 14. 86 is a composite number.
# 94 is divisible by 2, 7, and 14. 94 is a composite number.
# 96 is divisible by 2, 4, 8, 16, 32, and 64. So 96 is not a prime number.

In [237]:
import numpy as np

acc_df = pd.DataFrame(columns=["w/o cot", "cot"])

Difficulty_list = df['Difficulty'].unique()
for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue
    acc = np.array([0, 0])
    temp_df = df[df['Difficulty'] == Difficulty]
    
    without_cot_prevQustion = False
    with_cot_prevQustion = False
    
    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        if i==0:
            if row['Predict'] == row['Answer']:
                without_cot_prevQustion = True 
        if i==1:
            if row['Predict'] == row['Answer']:
                with_cot_prevQustion = True 
        if i==2:
            if without_cot_prevQustion:
                if row['Predict'] == row['Answer']:
                    acc[0] += 1
            without_cot_prevQustion = False
        if i==3:
            if with_cot_prevQustion:
                if row['Predict'] == row['Answer']:
                    acc[1] += 1  
            with_cot_prevQustion = False 
    acc = acc / (len(temp_df) // 4) * 100
    
    acc_df.loc[Difficulty] = acc
acc_df.round(2).reindex(['2_digit_composite',
                                                   '3_digit_easy_composite', 
                                                   '4_digit_easy_composite',
                                                   '3_digit_medium_composite',
                                                   '4_digit_medium_composite', 
                                                   '4_digit_hard_composite'])

,w/o cot,cot
2_digit_composite,97.10,82.61
3_digit_easy_composite,80.69,80.69
4_digit_easy_composite,98.80,99.60
3_digit_medium_composite,53.57,71.43
4_digit_medium_composite,93.00,98.60
4_digit_hard_composite,93.13,100.00


In [211]:
import numpy as np

acc_df = pd.DataFrame(columns=["w/o cot", "cot"])

Difficulty_list = df['Difficulty'].unique()
for Difficulty in Difficulty_list:
    if Difficulty[-9:] == "composite":
        continue
    acc = np.array([0, 0])
    temp_df = df[df['Difficulty'] == Difficulty]
    
    without_cot_prevQustion = False
    with_cot_prevQustion = False
    
    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        if i==0:
            if row['Predict'] == row['Answer']:
                without_cot_prevQustion = True 
        if i==1:
            if row['Predict'] == row['Answer']:
                with_cot_prevQustion = True 
        if i==2:
            if without_cot_prevQustion:
                if row['Predict'] == row['Answer']:
                    acc[0] += 1
            without_cot_prevQustion = False
        if i==3:
            if with_cot_prevQustion:
                if row['Predict'] == row['Answer']:
                    acc[1] += 1  
            with_cot_prevQustion = False                  
    acc = acc / (len(temp_df) // 4) * 100
    
    acc_df.loc[Difficulty] = acc
acc_df.round(2)

,w/o cot,cot
2_digit_prime,66.67,47.62
3_digit_prime,0.70,0.00
4_digit_prime,0.00,0.00


In [226]:
acc_df = pd.DataFrame(columns=["w/o cot", "cot"])

Difficulty_list = df['Difficulty'].unique()
acc = np.array([[0,  0], [0, 0], [0, 0]], dtype="float")
digit_len = np.array([0.0, 0.0, 0.0])

for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue    
    digit = int(Difficulty[0])
    temp_df = df[df['Difficulty'] == Difficulty]
    digit_len[digit-2] += (len(temp_df) // 4)
    without_cot_prevQustion = False
    with_cot_prevQustion = False
    
    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1
        if i==0:
            if row['Predict'] == row['Answer']:
                without_cot_prevQustion = True 
        if i==1:
            if row['Predict'] == row['Answer']:
                with_cot_prevQustion = True 
        if i==2:
            if without_cot_prevQustion:
                if row['Predict'] == row['Answer']:
                    acc[digit-2][0] += 1
            without_cot_prevQustion = False
        if i==3:
            if with_cot_prevQustion:
                if row['Predict'] == row['Answer']:
                    acc[digit-2][1] += 1
            with_cot_prevQustion = False
            
for i in range(3):
    acc[i] = acc[i] / digit_len[i] * 100
    acc_df.loc[f"{i+2}_digit_composite"] = acc[i]

acc_df.round(2)

,w/o cot,cot
2_digit_composite,97.10,82.61
3_digit_composite,74.81,78.68
4_digit_composite,95.58,99.20


In [231]:
pd.set_option('mode.chained_assignment',  None)

      
Difficulty_list = df['Difficulty'].unique()

undefined_df = pd.DataFrame(columns=["Qustion", "Output", "Predict", "Answer", "Difficulty" ,"Number", "Factor1", "Factor2"])
composite_cnt = 0

err_df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4"])
err_numbers = []

Difficulty_list = df['Difficulty'].unique()

right_composite = 0
wrong_composite = 0
for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue
    
    acc = np.array([0, 0, 0, 0])
    wrong_patton_acc = np.array([0, 0, 0, 0])
    
    temp_df = df[df['Difficulty'] == Difficulty]

    for idx, row in temp_df.iterrows():        
        i = int(idx[-1])-1
        if row['Predict'] == row['Answer']:
            acc[i] += 1
            if i==1 or i==3:
                right_composite += 1
        else: 
            continue
     
        fast_cut = False
        
        number_list = [3, 7, 11, 2, 5, 13, 17]
        for wrong_number in number_list:
            if (row['Number']%wrong_number)!= 0:
                s= f"{row['Number']} is divisible by {wrong_number}"
                if  row['Output'].find(s) != -1:
                    wrong_patton_acc[i] += 1
                    err_numbers.append(row['Number'])
                    wrong_composite += 1
                    fast_cut = True
                    break
                number_list = [3, 7, 11, 2, 5, 13, 17]
        
        if fast_cut:
            continue                
        
        for wrong_number in number_list:
            if (row['Number']%wrong_number)!= 0:
                s= f"{row['Number']} is also divisible by {wrong_number}"
                if  row['Output'].find(s) != -1:
                    wrong_patton_acc[i] += 1
                    wrong_composite += 1
                    err_numbers.append(row['Number'])
                    fast_cut = True
                    break
            
        if fast_cut:
            continue
        
        if row['Output'].rfind("is divisible by") != -1:
            cut_output = row['Output'][row['Output'].rfind("is divisible by")+len("is divisible by")+1:]
            cut_output = cut_output.split(".")[0]
            cut_output = cut_output.replace("and", ",")
            cut_output = cut_output.replace(" , ", "") 
            cut_output = cut_output.replace("itself", f"{row['Number']}")
            
            
            try: 
                num_list = list(map(int, cut_output.split(",")))
            except:
                if len(cut_output) >= 3:
                    if cut_output[-3] == ",":
                        cut_output = cut_output[:-5]
                        num_list = list(map(int, cut_output.split(",")))
            
            try:
                # num_list = list(map(int, cut_output.split(",")))
                for factor in num_list:
                    if (row['Number']%factor)==0:
                        continue
                    else:
                        fast_cut = True
                        wrong_composite += 1
                        err_numbers.append(row['Number'])
                        break 
            except:
                if i==1 or i==3:
                    undefined_df.loc[f"{row['Number']}_Q{i+1}"] = row  
    
        if fast_cut:
            continue 
                    
        if i==1 or i==3:
            # Factors
            for prime in primes:
                if (row['Number']%prime) == 0:
                    factor1 = prime 
                    factor2 = row['Number']//prime
                    break
            undefined_df.loc[f"{row['Number']}_Q{i+1}"] = row
            undefined_df.loc[f"{row['Number']}_Q{i+1}", 'Factor1'] = factor1
            undefined_df.loc[f"{row['Number']}_Q{i+1}", 'Factor2'] = factor2
            
    # err_df.loc[Difficulty] = wrong_patton_acc / acc * 100
    err_df.loc[Difficulty] = wrong_patton_acc
    
    
# err_df = err_df.drop(["Q1", "Q3"], axis=1)
err_df.round(2).reindex(['2_digit_composite',
                        '3_digit_easy_composite', 
                        '4_digit_easy_composite',
                        '3_digit_medium_composite',
                        '4_digit_medium_composite', 
                        '4_digit_hard_composite'])

,Q1,Q2,Q3,Q4
2_digit_composite,0,0,0,3
3_digit_easy_composite,0,60,0,83
4_digit_easy_composite,0,301,0,305
3_digit_medium_composite,0,38,0,47
4_digit_medium_composite,0,436,0,487
4_digit_hard_composite,0,124,0,131


In [234]:
err_numbers.append(301)

In [235]:
err_numbers[:4]

[10, 10, 14, 14]

In [236]:
import numpy as np

acc_df = pd.DataFrame(columns=["cot"])
err_df = pd.DataFrame(columns=["cot"])

Difficulty_list = df['Difficulty'].unique()
for Difficulty in Difficulty_list:
    if Difficulty[-5:] == "prime":
        continue
    acc = np.array([0])
    err = np.array([0])
    temp_df = df[df['Difficulty'] == Difficulty]
    
    without_cot_prevQustion = False
    with_cot_prevQustion = False
    
    for idx, row in temp_df.iterrows():
        i = int(idx[-1])-1

        if i==1:
            if row['Predict'] == row['Answer']:
                with_cot_prevQustion = True 
        if i==3:
            if with_cot_prevQustion:
                if row['Predict'] == row['Answer']:
                    acc[0] += 1  
                    if row['Number'] in err_numbers:
                        err[0] += 1
            with_cot_prevQustion = False 
    acc =  err / acc * 100
    
    acc_df.loc[Difficulty] = acc
acc_df.round(2)

,cot
2_digit_composite,40.35
3_digit_easy_composite,95.09
3_digit_medium_composite,100.00
4_digit_easy_composite,96.79
4_digit_hard_composite,100.00
4_digit_medium_composite,99.59
